In [12]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)


def is_flagged_word(description):
    flagged_words = [
                    'online','official','call','fantasy','contact','installation', 
                    'pay', 'manufacturer', 'delivery', 'established', 'registration',
                    'seo', 'service', 'customers', 'trading', 'company', 'buy', 'sell', 
                    'escort', 'provider', 'product', 'patients', 'website', 'sites', 
                    'free', 'affordable', 'website', 'buisness', 'installed', 'install',
                    'management', 'manufacture', 'install', 'download', 'provides',
                    'downloaded', 'services', 'development', 'drug', 'top', 'reviews',
                    'review', 'treatment', 'deliver', 'hire', 'hiring', 'supply', 'supplier', 
                    'commercial', 'discount', 'account', 'websites', 'app', 'products', 
                    'guests', 'deals'
                    ]
     
    return description.lower() in flagged_words


def contains_flagged_word(description):
    for user in df:
        words = re.split('\W+', description)
        for word in words:
            if(is_flagged_word(word)):
                return True
            
    
         
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)
df.head()
df["contains_flagged_word"] = df["description"].map(contains_flagged_word)  
df_potential_bot = df[
    ((df["edits"] + df["comments"] + df["reviews"] + df["taxonomy"] == 0) & df["contains_flagged_word"])
].copy()
df_potential_bot.sort_values(["id"], inplace=True)
df_potential_bot
df_potential_bot.to_csv('all_potential_bots.csv', index=False)

/Users/kellyshor/.asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
